## 1 **Factiva Article Dataset Construction**

#### 1.1 **Purpose:** This code takes html-formatted articles from Factiva search results and iteratively converts the html to pandas dataframes and then appends each dataframe of articles (df2) to a mother dataframe (df). It also shows post-hoc clean up of df, such as removing duplicate articles and oddly formatted quotation marks. The finished df is then fed into a different Python project that extracts and attributes quotes and paraphrases from decisions makers within these articles. 

#### 1.2 **Input data:** I took the list of firms from my corporate dataset from Pitchbook and filtered out all the automakers. I then created Factiva search quiries with the following steps: 
##### Step 1 - I took the base "Executive quotes" search code and removed the XXX filters and added some "said equivalents" based on YYY and add varients of the name of the focal automaker as well as the date range of 1/1/90 to 3/X/2021.
##### Step 2 - I ran these searches for each automaker and saw how many hits were in each year. 
##### Step 3 - Factiva does not let you scrape their website. It also only lets you download a report of 100 articles at a time so I had to carefully go through each firm-year with over 100 articles and break up search date ranges to get as close to 100 as possible. I also combined years to equal as close to 100 as possible. Ultimately there were over 5,000 searches. 
##### Step 4 - I then ran each of these 5,000+ searches manually, downloading each search result in RTF format. As part of this manual process I would run a Python code to move the downloaded file and rename it to a folder of choice. I did this with two everything (screens, Factiva, tracking, renaming Python code, etc.) to speed up the process. 
##### Step 5 - I reran Step 1 - 4 but changing the roles references to middle manager-type titles. 
##### Step 6 - I merged the articles from Step 4 and Step 5 to arrive at 270,193 unique articles (about 70K more than with the initial Step 1-4) 



## 2.0: Results 

### 2.1: Set up environment

In [2]:
import time
from tkinter import *
import datetime
import os
import glob
import timeit
import striprtf
import PyRTF
import glob
import re
import sqlite3
import csv
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import pickle as pkl


### 2.2: Run code on htmls

#### This code converts the html file into a pandas dataset with each article creating one row. While the specific results below are from articles with specific reference to middle manager titles, the same code was used on the initial round of Factiva querries.  

In [4]:
# Set the path to the folder you'd like to use:
path = 'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/htmls2/*'

fields = ['AN', 'SE', 'HD', 'CR', 'WC', 'PD', 'ET', 'SN', 'SC', 'ED', 'PG', 'LA', 'CY', 'LP', 'TD', 'CT', 'RF', 'CO',
          'IN', 'NS', 'RE', 'IPC', 'IPD', 'PUB']

df = pd.DataFrame(columns=fields)

t0 = time.time()
n = 0
for f in glob.glob(path + "*.html"):
    t1 = time.time()
    name = f.split('\\')
    name = name[1]
    file_div = name.split('_')
    firm = file_div[0]
    vertexid = file_div[1]
    s = file_div[2]
    e = file_div[3].split('.')[0]

    df1 = pd.read_html(f, index_col=0)
    df2 = pd.concat([l for l in df1 if 'HD' in l.index.values], axis=1).T
    df2['Firm'] = firm
    df2['vertex.id'] = vertexid
    df2['start'] = s
    df2['end'] = e
    df = df.append(df2, ignore_index=True)
    n += 1
    t2 = time.time()
    total = t1 - t0
    print(n, firm, vertexid, s, e, "time run:", round(t2-t1,2), "total hours:", round((t2-t0)/(60*60),2), "mean rate:", round((t2-t0)/n,2))

print("done")


1 MM ACCars 12 19900101 time run: 0.04 total hours: 0.0 mean rate: 0.09
2 MM Acura 20 19900101 time run: 0.74 total hours: 0.0 mean rate: 0.41
3 MM Acura 20 20000101 time run: 1.02 total hours: 0.0 mean rate: 0.62
4 MM Acura 20 20040101 time run: 1.02 total hours: 0.0 mean rate: 0.72
5 MM Acura 20 20070101 time run: 0.72 total hours: 0.0 mean rate: 0.72
6 MM Acura 20 20100101 time run: 0.99 total hours: 0.0 mean rate: 0.76
7 MM Acura 20 20140101 time run: 0.92 total hours: 0.0 mean rate: 0.79
8 MM Acura 20 20180101 time run: 0.63 total hours: 0.0 mean rate: 0.77
9 MM AM 106 20200101 time run: 0.28 total hours: 0.0 mean rate: 0.71
10 MM AM 39 19900101 time run: 0.06 total hours: 0.0 mean rate: 0.65
11 MM Aptera 83 19900101 time run: 0.06 total hours: 0.0 mean rate: 0.6
12 MM Arrinera 93 19900101 time run: 0.03 total hours: 0.0 mean rate: 0.55
13 MM Audi 112 19900101 time run: 0.93 total hours: 0.0 mean rate: 0.58
14 MM Audi 112 19960101 time run: 0.75 total hours: 0.0 mean rate: 0.59
15

## 3.0 **Review**
### These is the df of all articles from the middle manager Factiva search querries

In [5]:
df

,AN,SE,HD,CR,WC,PD,ET,SN,SC,ED,...,PUB,Firm,vertex.id,start,end,BY,ART,VOL,CLM,CX
0,Document TRENDE0020071008e3a80035x,NaN,Ford FPV GT Cobra (AU),NaN,310 words,8 October 2007,NaN,Trend News Agency (Azerbaijan),TRENDE,NaN,...,Trend Agency,MM,ACCars,12,19900101,NaN,NaN,NaN,NaN,NaN
1,Document prwkus0020010909dvbf001ym,NaN,"Ketchum plucks Mattel pro to head up LA, Sacra...",NaN,299 words,15 November 1999,NaN,PR Week US,PRWKUS,NaN,...,NaN,MM,Acura,20,19900101,By AIMEE GROVE.,NaN,NaN,NaN,NaN
2,Document finp000020010827dvbc00jxv,Driver's Edge,"Car sales booming, dealers report",Southam News,452 words,12 November 1999,NaN,National Post,FINP,National !@SERIES=The Weekly Auto Report,...,NaN,MM,Acura,20,19900101,Jennifer Campbell,NaN,NaN,NaN,NaN
3,Document jiji000020010906dvb900dbr,NaN,Honda Hobbled by Yen Strength in 1st Half,NaN,304 words,9 November 1999,09:24 GMT,Jiji Press English News Service,JIJI,NaN,...,NaN,MM,Acura,20,19900101,NaN,NaN,NaN,NaN,NaN
4,Document krtbn00020010828dvah01wdj,NaN,Chicago Tribune Auto Letters Column,NaN,916 words,17 October 1999,NaN,KRTBN Knight-Ridder Tribune Business News: Chi...,KRTBN,NaN,...,NaN,MM,Acura,20,19900101,Jim Mateja,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146947,Document AIWFIE0020091030e5av00006,FE MotoBahn,'We plan to concentrate on the Rio this financ...,NaN,512 words,31 October 2009,NaN,Financial Express (India),AIWFIE,NaN,...,India Express Mumbai Limited,MM,Zotye,1801,19900101,NaN,NaN,NaN,NaN,NaN
146948,Document WDAN000020091029e5at0001v,NaN,Premier Strikes Import Deal with China's Zotye...,NaN,297 words,29 October 2009,NaN,IHS Global Insight Daily Analysis,WDAN,NaN,...,Global Insight Limited,MM,Zotye,1801,19900101,Paul Newton,NaN,NaN,NaN,NaN
146949,Document HNASNI0020091029e5at000e1,NaN,Premier Automobiles re-enters passenger car se...,NaN,323 words,29 October 2009,NaN,Asian News International,HNASNI,NaN,...,HT Media Limited,MM,Zotye,1801,19900101,NaN,NaN,NaN,NaN,NaN
146950,Document AIWINE0020091029e5at00018,Business,premier drives back with suv,NaN,249 words,29 October 2009,NaN,Indian Express,AIWINE,NaN,...,India Express Mumbai Limited,MM,Zotye,1801,19900101,Express news service,NaN,NaN,NaN,NaN


### Download the df to pkl and csv 

In [6]:
df.to_csv(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/full_dataMM.csv')

In [7]:
df.to_pickle(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/full_dataMM.pkl')

### Combine middle managers-specific articles with general articles

In [8]:
df1 = pd.read_pickle(r'C:\Users\danwilde\Dropbox (Penn)\Dissertation\Factiva\full_data.pkl')

In [9]:
frames = [df, df1]

In [10]:
dffull = pd.concat(frames)

In [11]:
dffull

,AN,SE,HD,CR,WC,PD,ET,SN,SC,ED,...,PUB,Firm,vertex.id,start,end,BY,ART,VOL,CLM,CX
0,Document TRENDE0020071008e3a80035x,NaN,Ford FPV GT Cobra (AU),NaN,310 words,8 October 2007,NaN,Trend News Agency (Azerbaijan),TRENDE,NaN,...,Trend Agency,MM,ACCars,12,19900101,NaN,NaN,NaN,NaN,NaN
1,Document prwkus0020010909dvbf001ym,NaN,"Ketchum plucks Mattel pro to head up LA, Sacra...",NaN,299 words,15 November 1999,NaN,PR Week US,PRWKUS,NaN,...,NaN,MM,Acura,20,19900101,By AIMEE GROVE.,NaN,NaN,NaN,NaN
2,Document finp000020010827dvbc00jxv,Driver's Edge,"Car sales booming, dealers report",Southam News,452 words,12 November 1999,NaN,National Post,FINP,National !@SERIES=The Weekly Auto Report,...,NaN,MM,Acura,20,19900101,Jennifer Campbell,NaN,NaN,NaN,NaN
3,Document jiji000020010906dvb900dbr,NaN,Honda Hobbled by Yen Strength in 1st Half,NaN,304 words,9 November 1999,09:24 GMT,Jiji Press English News Service,JIJI,NaN,...,NaN,MM,Acura,20,19900101,NaN,NaN,NaN,NaN,NaN
4,Document krtbn00020010828dvah01wdj,NaN,Chicago Tribune Auto Letters Column,NaN,916 words,17 October 1999,NaN,KRTBN Knight-Ridder Tribune Business News: Chi...,KRTBN,NaN,...,NaN,MM,Acura,20,19900101,Jim Mateja,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387781,Document TAIP000020171127edbr0000q,NaN,Yulon Nissan to launch three car models next ...,Staff reporter,345 words,27 November 2017,NaN,Taipei Times,TAIP,NaN,...,Liberty Times Ltd.,Yulon,1774,20170101,20211231,By Kuo Chia-erh,NaN,NaN,NaN,NaN
387782,Document TAIP000020170518ed5i00003,NaN,Yulon announces launch of Luxgen SUV next qua...,Staff reporter,430 words,18 May 2017,NaN,Taipei Times,TAIP,NaN,...,Liberty Times Ltd.,Yulon,1774,20170101,20211231,By Kuo Chia-erh,NaN,NaN,NaN,NaN
387783,Document TAIP000020170513ed5d00005,NaN,Yulon Nissan cash dividend soars on high earn...,Staff reporter,517 words,13 May 2017,NaN,Taipei Times,TAIP,NaN,...,Liberty Times Ltd.,Yulon,1774,20170101,20211231,By Kuo Chia-erh,NaN,NaN,NaN,NaN
387784,Document TAIP000020170414ed4e00001,NaN,Yulon Nissan sets high profit target amid cos...,Staff reporter,410 words,14 April 2017,NaN,Taipei Times,TAIP,NaN,...,Liberty Times Ltd.,Yulon,1774,20170101,20211231,By Kuo Chia-erh,NaN,NaN,NaN,NaN


In [12]:
dffull.to_pickle(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/full_dataMM_all.pkl')

### Remove duplicates

In [1]:
import time
from tkinter import *
import datetime
import os
import glob
import timeit
import striprtf
import PyRTF
import glob
import re
import sqlite3
import csv
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import _pickle 
import time

In [2]:
df = pd.read_pickle(r'C:\Users\danwilde\Dropbox (Penn)\Dissertation\Factiva\full_dataMM_all.pkl')

In [3]:
df1 = df.drop_duplicates(subset='AN', keep="first")

In [4]:
df1

,AN,SE,HD,CR,WC,PD,ET,SN,SC,ED,...,PUB,Firm,vertex.id,start,end,BY,ART,VOL,CLM,CX
0,Document TRENDE0020071008e3a80035x,NaN,Ford FPV GT Cobra (AU),NaN,310 words,8 October 2007,NaN,Trend News Agency (Azerbaijan),TRENDE,NaN,...,Trend Agency,MM,ACCars,12,19900101,NaN,NaN,NaN,NaN,NaN
1,Document prwkus0020010909dvbf001ym,NaN,"Ketchum plucks Mattel pro to head up LA, Sacra...",NaN,299 words,15 November 1999,NaN,PR Week US,PRWKUS,NaN,...,NaN,MM,Acura,20,19900101,By AIMEE GROVE.,NaN,NaN,NaN,NaN
2,Document finp000020010827dvbc00jxv,Driver's Edge,"Car sales booming, dealers report",Southam News,452 words,12 November 1999,NaN,National Post,FINP,National !@SERIES=The Weekly Auto Report,...,NaN,MM,Acura,20,19900101,Jennifer Campbell,NaN,NaN,NaN,NaN
3,Document jiji000020010906dvb900dbr,NaN,Honda Hobbled by Yen Strength in 1st Half,NaN,304 words,9 November 1999,09:24 GMT,Jiji Press English News Service,JIJI,NaN,...,NaN,MM,Acura,20,19900101,NaN,NaN,NaN,NaN,NaN
4,Document krtbn00020010828dvah01wdj,NaN,Chicago Tribune Auto Letters Column,NaN,916 words,17 October 1999,NaN,KRTBN Knight-Ridder Tribune Business News: Chi...,KRTBN,NaN,...,NaN,MM,Acura,20,19900101,Jim Mateja,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387732,Document CM00000020100209e62900044,NaN,MARKET TALK: Thai Shares Off 0.5% Midday; Sup...,NaN,"1,630 words",9 February 2010,05:48 GMT,Dow Jones Capital Markets Report,CM,NaN,...,"Dow Jones & Company, Inc.",Yulon,1774,20070101,20161231,NaN,NaN,NaN,NaN,NaN
387733,Document CM00000020100126e61q00022,NaN,"MARKET TALK: NZD/USD Range Trading, Eyes On F...",NaN,"1,401 words",26 January 2010,03:01 GMT,Dow Jones Capital Markets Report,CM,NaN,...,"Dow Jones & Company, Inc.",Yulon,1774,20070101,20161231,NaN,NaN,NaN,NaN,NaN
387746,Document TAIP000020090219e52j00008,NaN,SIP confirms Wu Shu-jen submitted donation li...,STAFF REPORTER,468 words,19 February 2009,NaN,Taipei Times,TAIP,NaN,...,Liberty Times Ltd,Yulon,1774,20070101,20161231,By Rich Chang,NaN,NaN,NaN,NaN
387764,Document CNAENG0020210220eh2k000be,NaN,Hon Hai's open electric car platform to launc...,NaN,648 words,20 February 2021,NaN,Central News Agency English News,CNAENG,NaN,...,Central News Agency Hong Kong Bureau,Yulon,1774,20170101,20211231,NaN,NaN,NaN,NaN,NaN


In [5]:
df1.to_pickle(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/filtered_data.pkl')

### Explore flagging articles that are transcripts. I ended up doing a version of this in my quotes extraction code so this is more exploratory than anything

In [6]:
df = pd.read_pickle(r'C:\Users\danwilde\Dropbox (Penn)\Dissertation\Factiva\filtered_data.pkl')

In [7]:
df_c1 = df.loc[df['NS'].str.contains("c151", case=False, na=False)]

In [8]:
df_c2 = df_c1.loc[df_c1['NS'].str.contains("ntra : Transcripts", case=False, na=False)]

In [9]:
df_c1 = df.loc[df['NS'].str.contains("ntra : Transcripts", case=False, na=False)]

In [10]:
df_c2 = df_c1.loc[df_c1['NS'].str.contains("c151", case=False, na=False)]

In [11]:
df_c1.to_excel(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/df_c1.xlsx')

In [12]:
df_c2

,AN,SE,HD,CR,WC,PD,ET,SN,SC,ED,...,PUB,Firm,vertex.id,start,end,BY,ART,VOL,CLM,CX
2070,Document FNDW000020100902e68i0002v,NaN,Preliminary 2010 Brambles Limited Earnings Pre...,NaN,12314 words,18 August 2010,NaN,CQ FD Disclosure,FNDW,NaN,...,"CQ-Roll Call, Inc.",MM,Audi,112,20100702,NaN,NaN,NaN,NaN,NaN
2730,Document FNDW000020140315ea3d004s9,NaN,Event Brief of Full Year 2013 Volkswagen AG Ea...,NaN,13827 words,13 March 2014,NaN,CQ FD Disclosure,FNDW,NaN,...,"CQ-Roll Call, Inc.",MM,Audi,112,20140128,NaN,NaN,NaN,NaN,NaN
2798,Document AFINCC0020140723ea7n00003,NaN,KPIT Technologies - Analyst Meet - For June 20...,NaN,6728 words,23 July 2014,NaN,Accord Fintech: Indian Company Con-Calls,AFINCC,NaN,...,Accord Fintech,MM,Audi,112,20140617,NaN,NaN,NaN,NaN,NaN
3201,Document AFINCC0020160817ec8a000dx,NaN,Dynamatic Technologies - Analyst Meet - For Ju...,NaN,3297 words,10 August 2016,NaN,Accord Fintech: Indian Company Con-Calls,AFINCC,NaN,...,Accord Fintech,MM,Audi,112,20160701,NaN,NaN,NaN,NaN,NaN
3338,Document FNDW000020180622ee6l002p9,NaN,Universal Display Corp Annual Shareholders Mee...,NaN,4219 words,21 June 2018,NaN,CQ FD Disclosure,FNDW,NaN,...,"CQ-Roll Call, Inc.",MM,Audi,112,20180101,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364346,Document FNDW000020200206eg250038p,NaN,Q4 2019 ALKAbello A/S Earnings Call - Final,NaN,"7,662 words",5 February 2020,NaN,CQ FD Disclosure,FNDW,NaN,...,"CQ-Roll Call, Inc.",Volvo,1709,20200104,20200404,NaN,NaN,NaN,NaN,NaN
364505,Document FNDW000020200717eg7g001uq,NaN,Q2 2020 Husqvarna AB Earnings Call - Final,NaN,"10,045 words",16 July 2020,NaN,CQ FD Disclosure,FNDW,NaN,...,ASC Services LLC,Volvo,1709,20200706,20201005,NaN,NaN,NaN,NaN,NaN
381649,Document DEUEN00020150423eb4n000b5,NaN,VW power struggle enters new round,NaN,274 words,23 April 2015,11:12 GMT,Deutsche Welle,DEUEN,NaN,...,Deutsche Welle,VW,1705,20150423,20150508,NaN,NaN,NaN,NaN,NaN
382647,Document FNDW000020151120ebbj001p5,NaN,Half Year 2015 Johnson Matthey PLC Earnings P...,NaN,"10,859 words",19 November 2015,NaN,CQ FD Disclosure,FNDW,NaN,...,"CQ-Roll Call, Inc.",VW,1705,20151117,20151202,NaN,NaN,NaN,NaN,NaN


In [13]:
df['c151'] = np.where(df['NS'].str.contains('c151',case=False, na=False),1,0)

In [14]:
df['ntra'] = np.where(df['NS'].str.contains('ntra\s*\:\s*Transcripts',case=False, na=False),1,0)

In [15]:
df['ops'] = np.where(df['LP'].str.contains('OPERATOR:',case=False, na=False),1,0)

In [16]:
df['parts'] = np.where(df['LP'].str.contains('CORPORATE  PARTICIPANTS',case=False, na=False),1,0)

In [17]:
df['pres'] = np.where(df['LP'].str.contains('^Presentation',case=False, na=False),1,0)

In [18]:
df_pres = df.loc[(df['pres']==1)]

In [19]:
df_pres.to_excel(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/df_pres.xlsx')

In [20]:
df_ntra = df.loc[(df['ntra']==1) & (df['c151']==1)]

In [21]:
df['exclude'] = ((df['ntra']==1) & (df['c151']==1))| (df['ops'] == 1) | (df['parts'] == 1) | (df['pres'] == 1)

In [22]:
#Convert Booleon (T/F) to 0/1
df["exclude"] = df["exclude"].astype(int)

In [23]:
#See how many articles fall into the conference call etc. camp
df3 = df.loc[(df['exclude']==1)]

In [37]:
df3

,AN,SE,HD,CR,WC,PD,ET,SN,SC,ED,...,ART,VOL,CLM,CX,c151,ntra,ops,parts,pres,exclude
91,Document FNDW000020030730dz7l004m8,NaN,"Q2 2003 BorgWarner, Inc. Earnings Conference C...",NaN,8053 words,21 July 2003,NaN,FD (FAIR DISCLOSURE) WIRE,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,0,1,0,0,1
364,Document FNDW000020120301e82g002p9,NaN,Q4 2011 General Motors Company Earnings Confer...,NaN,12100 words,16 February 2012,NaN,CQ FD Disclosure,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,0,1,0,1,1
377,Document FNDW000020110825e789007bx,NaN,General Motors 2nd Annual Global Business Conf...,NaN,29383 words,9 August 2011,NaN,CQ FD Disclosure,FNDW,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,1,1
1058,Document FNDW000020030804dz7o001vu,NaN,Q3 2003 Microsemi Earnings Conference Call - F...,NaN,5271 words,24 July 2003,NaN,FD (FAIR DISCLOSURE) WIRE,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,0,1,0,0,1
1195,Document FNDW000020031110dzar000ji,NaN,"Q3 2003 BorgWarner, Inc. Earnings Conference C...",NaN,7144 words,27 October 2003,NaN,FD (FAIR DISCLOSURE) WIRE,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378960,Document FNDW000020120301e82g001up,NaN,Q4 2011 Readsoft AB Earnings Conference Call ...,NaN,"4,496 words",16 February 2012,NaN,CQ FD Disclosure,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,0,0,0,1,1
381649,Document DEUEN00020150423eb4n000b5,NaN,VW power struggle enters new round,NaN,274 words,23 April 2015,11:12 GMT,Deutsche Welle,DEUEN,NaN,...,NaN,NaN,NaN,NaN,1,1,0,0,0,1
382647,Document FNDW000020151120ebbj001p5,NaN,Half Year 2015 Johnson Matthey PLC Earnings P...,NaN,"10,859 words",19 November 2015,NaN,CQ FD Disclosure,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,1,0,0,1,1
386855,Document FNDW000020200401eg3v0040h,NaN,Q4 2019 Mahle Metal Leve SA Earnings Call - F...,NaN,"4,471 words",31 March 2020,NaN,CQ FD Disclosure,FNDW,NaN,...,NaN,NaN,NaN,NaN,1,1,1,0,1,1


In [25]:
df3.to_excel(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/df3.xlsx')

### Replace the non-Utf8 formatted quotes with conventional ones 

In [26]:
df = pd.read_pickle(r'C:\Users\danwilde\Dropbox (Penn)\Dissertation\Factiva\filtered_data.pkl')

In [27]:
df['LP'] = df['LP'].str.replace('“','"')
df['TD'] = df['TD'].str.replace('“','"')

In [28]:
df['LP'] = df['LP'].str.replace('”','"')
df['TD'] = df['TD'].str.replace('”','"')

### Reset the index for the main dataset and create final df of articles

In [29]:
df = df.reset_index(drop=False)

In [30]:
df.rename(columns = {'index':'old index'}, inplace = True) 

In [33]:
df.to_pickle(r'C:/Users/danwilde/Dropbox (Penn)/Dissertation/Factiva/filtered_data_full 2021_09_17.pkl')

In [34]:
df = pd.read_pickle(r'C:\Users\danwilde\Dropbox (Penn)\Dissertation\Factiva\filtered_data_full 2021_09_17.pkl')